In [35]:
import random
from collections import defaultdict, Counter

# Directions and patterns
directions = ['R', 'U', 'L']
false_answers = {
    'Am': [
        ['R', 'U', 'L', 'R', 'R'],
        ['U', 'R'],
        ['R', 'U', 'R', 'R', 'U', 'R']
    ],
    'Club': [
        ['U', 'U', 'L', 'L', 'R', 'R'],
        ['L', 'U', 'R', 'R', 'U', 'U', 'R', 'L', 'R'],
        ['U', 'R']
    ],
    'Premier': [
        ['R', 'U', 'R', 'R', 'R', 'U', 'U', 'L', 'U'],
        ['U', 'U', 'U', 'L', 'U', 'U', 'U', 'L', 'U'],
        ['L', 'U', 'L', 'U', 'U', 'L', 'U', 'U', 'L'],
        ['R', 'U', 'L', 'R', 'R', 'R', 'U', 'U', 'L'],
        ['L', 'L', 'R', 'U', 'U', 'L', 'U', 'R', 'R'],
        ['R', 'R', 'R', 'R', 'R', 'U', 'L']
    ]
}

# Flatten transitions from all levels
def extract_transitions(patterns):
    transitions = []
    for sequence in patterns:
        for i in range(len(sequence) - 1):
            transitions.append((sequence[i], sequence[i + 1]))
    return transitions

# Build Markov model
def build_markov_model(transitions):
    model = defaultdict(lambda: Counter())
    for from_dir, to_dir in transitions:
        model[from_dir][to_dir] += 1
    for from_dir in model:
        total = sum(model[from_dir].values())
        for to_dir in model[from_dir]:
            model[from_dir][to_dir] /= total
    return model

# Predict the next most likely false answer
def predict_next_false(model, last):
    if last not in model or not model[last]:
        return random.choice(directions)
    options = list(model[last].keys())
    probs = list(model[last].values())
    return random.choices(options, weights=probs, k=1)[0]

# Simulate the game with Markov-based strategy
def simulate_markov_game(false_patterns, rounds=10, simulations=10000):
    flat_false = [item for sublist in false_patterns for item in sublist]
    transitions = extract_transitions(false_patterns)
    model = build_markov_model(transitions)

    win_count = 0

    for _ in range(simulations):
        success = True
        last_false = random.choice(directions)

        for _ in range(rounds):
            actual_false = random.choice(flat_false)
            predicted_false = predict_next_false(model, last_false)
            safe_choices = [d for d in directions if d != predicted_false]

            # Assume correct answer is one of the safe ones
            correct_answer = random.choice([d for d in directions if d != actual_false])
            player_choice = random.choice(safe_choices)

            if player_choice != correct_answer:
                success = False
                break

            last_false = actual_false

        if success:
            win_count += 1

    return round(win_count / simulations, 4)

# Run the simulation for each difficulty
results = {}
for level, patterns in false_answers.items():
    win_prob = simulate_markov_game(patterns)
    results[level] = win_prob

# Print results
print("🎯 Win Probability Using Markov-Based Prediction Strategy (10 Rounds in a Row):")
for level, prob in results.items():
    print(f"{level}: {prob * 100:.2f}%")


🎯 Win Probability Using Markov-Based Prediction Strategy (10 Rounds in a Row):
Am: 0.02%
Club: 0.00%
Premier: 0.00%


# Task
Create a user interface using `ipywidgets` with tabs for different difficulty levels, input fields for false patterns, and an output area for the simulated next pattern. Modify the existing Python code to accept input from the UI and display the simulation results in the output area.

## Setup ui

### Subtask:
Create a user interface using `ipywidgets` with tabs for different difficulty levels, input fields for false patterns, and an output area for the simulated next pattern.


**Reasoning**:
Create the ipywidgets for the user interface as described in the instructions, including tabs for difficulty levels, text areas for inputting patterns, a button to trigger the simulation, and an output area.



In [36]:
from ipywidgets import Tab, Textarea, Output, VBox, Button, display

output_area = Output()

input_widgets = {}
for level, patterns in false_answers.items():
    # Format patterns as a string with newlines for display in Textarea
    pattern_string = "\n".join([",".join(p) for p in patterns])
    input_widgets[level] = Textarea(
        value=pattern_string,
        description=f'{level} Patterns:',
        layout={'width': 'auto', 'height': '150px'}
    )

tab_children = []
tab_titles = []
for level in ['Am', 'Club', 'Premier']:
    tab_children.append(VBox([input_widgets[level]]))
    tab_titles.append(level)

tab_widget = Tab(children=tab_children)
for i, title in enumerate(tab_titles):
    tab_widget.set_title(i, title)

run_button = Button(description="Run Simulation")

ui = VBox([tab_widget, run_button, output_area])
display(ui)

ImportError: cannot import name 'display' from 'ipywidgets' (/usr/local/lib/python3.11/dist-packages/ipywidgets/__init__.py)

**Reasoning**:
The previous command failed because `display` is not available in `ipywidgets`. It should be imported from `IPython.display`. Correct the import statement and regenerate the code block to create and display the user interface.



In [ ]:
from ipywidgets import Tab, Textarea, Output, VBox, Button
from IPython.display import display

output_area = Output()

input_widgets = {}
for level, patterns in false_answers.items():
    # Format patterns as a string with newlines for display in Textarea
    pattern_string = "\n".join([",".join(p) for p in patterns])
    input_widgets[level] = Textarea(
        value=pattern_string,
        description=f'{level} Patterns:',
        layout={'width': 'auto', 'height': '150px'}
    )

tab_children = []
tab_titles = []
for level in ['Am', 'Club', 'Premier']:
    tab_children.append(VBox([input_widgets[level]]))
    tab_titles.append(level)

tab_widget = Tab(children=tab_children)
for i, title in enumerate(tab_titles):
    tab_widget.set_title(i, title)

run_button = Button(description="Run Simulation")

ui = VBox([tab_widget, run_button, output_area])
display(ui)

## Connect ui to logic

### Subtask:
Modify the existing Python code to accept input from the UI and display the simulation results in the output area.


**Reasoning**:
Define the function to handle the button click, extract the relevant false patterns from the UI based on the active tab, and link the function to the button's click event.



In [ ]:
def run_simulation(b):
    with output_area:
        output_area.clear_output()
        # Get the currently selected tab's index
        active_tab_index = tab_widget.selected_index
        # Get the title of the active tab (difficulty level)
        active_level = tab_widget.get_title(active_tab_index)

        # Get the Textarea widget for the active level
        active_textarea = input_widgets[active_level]

        # Get the pattern string from the Textarea and parse it
        pattern_string = active_textarea.value.strip()
        if not pattern_string:
            print(f"No patterns provided for {active_level} level.")
            return

        # Parse the patterns from the string
        patterns = []
        for line in pattern_string.split('\n'):
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            if pattern:
                patterns.append(pattern)

        if not patterns:
            print(f"Could not parse any valid patterns for {active_level} level.")
            return

        # Run the simulation with the parsed patterns
        win_prob = simulate_markov_game(patterns)

        # Display the results
        print("🎯 Win Probability Using Markov-Based Prediction Strategy (10 Rounds in a Row):")
        print(f"{active_level}: {win_prob * 100:.2f}%")

# Link the function to the button's on_click event
run_button.on_click(run_simulation)

## Refine simulation

### Subtask:
Adapt the `simulate_markov_game` function to work with the UI input and output, possibly simulating a single round or a smaller number of rounds for responsiveness.


**Reasoning**:
Review the existing `simulate_markov_game` function and determine if modifications are needed for better UI responsiveness. The current function simulates 10000 games of 10 rounds each, which is too slow for an interactive UI. I will modify the function to simulate a smaller, fixed number of rounds and games suitable for a responsive UI.



In [ ]:
def simulate_markov_game(false_patterns, rounds=5, simulations=100):
    """
    Simulates the game multiple times using a Markov-based prediction strategy.

    Args:
        false_patterns: A list of lists, where each inner list is a sequence
                        of false directions for a specific difficulty level.
        rounds: The number of rounds to play in each simulated game.
        simulations: The number of games to simulate.

    Returns:
        The win probability (as a float between 0 and 1) over all simulations.
    """
    flat_false = [item for sublist in false_patterns for item in sublist]
    if not flat_false: # Handle case with no false patterns
        return 0.0

    transitions = extract_transitions(false_patterns)
    model = build_markov_model(transitions)

    win_count = 0

    for _ in range(simulations):
        success = True
        # Start with a random direction as the 'last false' for the first round
        last_false = random.choice(directions)

        for _ in range(rounds):
            # If there are no false patterns, we can't pick an actual false answer
            if not flat_false:
                 actual_false = random.choice(directions) # Default to random if no patterns
            else:
                actual_false = random.choice(flat_false) # Pick a random false pattern

            predicted_false = predict_next_false(model, last_false)
            safe_choices = [d for d in directions if d != predicted_false]

            # If there are no safe choices (shouldn't happen with 3 directions and 1 prediction)
            if not safe_choices:
                 success = False
                 break

            # Assume correct answer is one of the safe ones (not the predicted false)
            # Also ensure the correct answer is not the actual false answer for this round
            possible_correct_answers = [d for d in directions if d != actual_false and d != predicted_false]

            # If there are no possible correct answers, the player cannot win this round
            if not possible_correct_answers:
                 success = False
                 break

            correct_answer = random.choice(possible_correct_answers)
            player_choice = random.choice(safe_choices)

            if player_choice != correct_answer:
                success = False
                break

            last_false = actual_false # Update last_false for the next round

        if success:
            win_count += 1

    return round(win_count / simulations, 4)

# Update the run_simulation function to use the modified simulate_markov_game
def run_simulation(b):
    with output_area:
        output_area.clear_output()
        active_tab_index = tab_widget.selected_index
        active_level = tab_widget.get_title(active_tab_index)
        active_textarea = input_widgets[active_level]
        pattern_string = active_textarea.value.strip()

        if not pattern_string:
            print(f"No patterns provided for {active_level} level.")
            return

        patterns = []
        for line in pattern_string.split('\n'):
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            if pattern:
                patterns.append(pattern)

        if not patterns:
            print(f"Could not parse any valid patterns for {active_level} level.")
            return

        # Use smaller numbers for UI responsiveness
        win_prob = simulate_markov_game(patterns, rounds=5, simulations=100)

        print(f"🎯 Win Probability Using Markov-Based Prediction Strategy ({active_level} - 5 Rounds, 100 Simulations):")
        print(f"{active_level}: {win_prob * 100:.2f}%")

# Link the function to the button's on_click event (already done in previous step, but re-linking for clarity)
run_button.on_click(run_simulation)

## Handle user input

### Subtask:
Implement logic to parse the user-provided patterns from the input fields.


**Reasoning**:
Implement the logic to parse the user-provided patterns from the input fields, including splitting lines and cleaning directions, and add error handling for empty or invalid input.



In [ ]:
def run_simulation(b):
    with output_area:
        output_area.clear_output()
        # Get the currently selected tab's index
        active_tab_index = tab_widget.selected_index
        # Get the title of the active tab (difficulty level)
        active_level = tab_widget.get_title(active_tab_index)

        # Get the Textarea widget for the active level
        active_textarea = input_widgets[active_level]

        # Get the pattern string from the Textarea
        pattern_string = active_textarea.value

        # Split the string into individual lines
        lines = pattern_string.strip().split('\n')

        patterns = []
        # Process each line
        for line in lines:
            # Split each line by commas and clean up whitespace
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            # Only add non-empty patterns
            if pattern:
                patterns.append(pattern)

        # Error handling for empty or invalid input
        if not patterns:
            print(f"No valid patterns provided for {active_level} level.")
            return

        # Run the simulation with the parsed patterns
        # Using smaller numbers for UI responsiveness
        win_prob = simulate_markov_game(patterns, rounds=5, simulations=100)

        # Display the results
        print(f"🎯 Win Probability Using Markov-Based Prediction Strategy ({active_level} - 5 Rounds, 100 Simulations):")
        print(f"{active_level}: {win_prob * 100:.2f}%")

# Link the function to the button's on_click event
# This link was already established, re-executing for clarity in this context
run_button.on_click(run_simulation)

## Update output

### Subtask:
Display the predicted next pattern(s) or simulation results in the designated output area of the UI.


**Reasoning**:
The subtask is to display the simulation results in the output area. The `run_simulation` function already calculates the win probability and prints it to the `output_area`. I will add a message indicating that the simulation is running when the button is clicked for better user feedback, clear the output area before displaying new results, print the win probability within a `with output_area:` block and format the output message as requested.



In [ ]:
def run_simulation(b):
    with output_area:
        output_area.clear_output() # 1. Ensure the output_area is cleared
        print("Simulation running...") # 4. Indicate simulation is running

        # Get the currently selected tab's index
        active_tab_index = tab_widget.selected_index
        # Get the title of the active tab (difficulty level)
        active_level = tab_widget.get_title(active_tab_index)

        # Get the Textarea widget for the active level
        active_textarea = input_widgets[active_level]

        # Get the pattern string from the Textarea
        pattern_string = active_textarea.value

        # Split the string into individual lines
        lines = pattern_string.strip().split('\n')

        patterns = []
        # Process each line
        for line in lines:
            # Split each line by commas and clean up whitespace
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            # Only add non-empty patterns
            if pattern:
                patterns.append(pattern)

        # Error handling for empty or invalid input
        if not patterns:
            print(f"No valid patterns provided for {active_level} level.")
            return

        # Run the simulation with the parsed patterns
        # Using smaller numbers for UI responsiveness
        num_rounds = 5
        num_simulations = 100
        win_prob = simulate_markov_game(patterns, rounds=num_rounds, simulations=num_simulations)

        # 2 & 3. Display the results in the output area with specified formatting
        output_area.clear_output() # Clear "Simulation running..." message
        print(f"🎯 Win Probability Using Markov-Based Prediction Strategy ({active_level} - {num_rounds} Rounds, {num_simulations} Simulations):")
        print(f"{active_level}: {win_prob * 100:.2f}%")

# Link the function to the button's on_click event
run_button.on_click(run_simulation)

## Summary:

### Data Analysis Key Findings

*   An `ipywidgets` UI was successfully created with tabs for 'Am', 'Club', and 'Premier' difficulty levels, `Textarea` widgets for inputting false patterns, a "Run Simulation" button, and an `Output` area.
*   The `run_simulation` function was implemented to handle button clicks, retrieve the active tab's difficulty level, parse user-provided false patterns from the corresponding `Textarea`, and run the simulation.
*   The `simulate_markov_game` function was adapted to accept parameters for the number of rounds and simulations, using smaller values (5 rounds, 100 simulations) for better UI responsiveness.
*   The simulation results, specifically the calculated win probability, are displayed in the `output_area` with clear formatting, including the difficulty level and simulation parameters.
*   Error handling was included to inform the user if no valid patterns are provided or parsed for a selected difficulty level.

### Insights or Next Steps

*   The current simulation assumes the "correct" answer is randomly chosen from the directions not predicted as false and not the actual false answer. A next step could involve exploring different assumptions about the correct answer's placement.
*   Consider adding input fields to the UI to allow users to adjust the number of rounds and simulations for the `simulate_markov_game` function, providing more control over the simulation's duration and precision.


# Task
Improve the visual interactivity of the Markov game simulation code to display results in real-time within the Jupyter environment.

## Refactor simulation for real-time updates

### Subtask:
Modify the `simulate_markov_game` function to yield results or update a visual element within the loop, allowing the user to see progress as the simulation runs. This might involve using `display` within the loop with a mechanism to overwrite the previous output.


**Reasoning**:
Modify the `simulate_markov_game` function to accept an output widget and update the output periodically within the simulation loop to show progress.



In [ ]:
def simulate_markov_game(false_patterns, rounds=5, simulations=100, output=None):
    """
    Simulates the game multiple times using a Markov-based prediction strategy,
    updating the output widget periodically.

    Args:
        false_patterns: A list of lists, where each inner list is a sequence
                        of false directions for a specific difficulty level.
        rounds: The number of rounds to play in each simulated game.
        simulations: The number of games to simulate.
        output: An ipywidgets.Output widget to display real-time progress.

    Returns:
        The final win probability (as a float between 0 and 1) over all simulations.
    """
    flat_false = [item for sublist in false_patterns for item in sublist]
    if not flat_false: # Handle case with no false patterns
        if output:
            with output:
                print("No false patterns provided. Win probability is 0.00%.")
        return 0.0

    transitions = extract_transitions(false_patterns)
    model = build_markov_model(transitions)

    win_count = 0

    for i in range(simulations):
        success = True
        # Start with a random direction as the 'last false' for the first round
        last_false = random.choice(directions)

        for _ in range(rounds):
            # If there are no false patterns, we can't pick an actual false answer
            if not flat_false:
                 actual_false = random.choice(directions) # Default to random if no patterns
            else:
                actual_false = random.choice(flat_false) # Pick a random false pattern

            predicted_false = predict_next_false(model, last_false)
            safe_choices = [d for d in directions if d != predicted_false]

            # If there are no safe choices (shouldn't happen with 3 directions and 1 prediction)
            if not safe_choices:
                 success = False
                 break

            # Assume correct answer is one of the safe ones (not the predicted false)
            # Also ensure the correct answer is not the actual false answer for this round
            possible_correct_answers = [d for d in directions if d != actual_false and d != predicted_false]

            # If there are no possible correct answers, the player cannot win this round
            if not possible_correct_answers:
                 success = False
                 break


            correct_answer = random.choice(possible_correct_answers)
            player_choice = random.choice(safe_choices)

            if player_choice != correct_answer:
                success = False
                break

            last_false = actual_false # Update last_false for the next round

        if success:
            win_count += 1

        # Update output periodically
        if output and (i + 1) % (simulations // 10 or 1) == 0 or (i + 1) == simulations:
             with output:
                output.clear_output(wait=True) # Clear previous output
                current_prob = round(win_count / (i + 1), 4)
                print(f"Simulations complete: {i + 1} / {simulations}")
                print(f"Current Win Probability: {current_prob * 100:.2f}%")


    final_win_prob = round(win_count / simulations, 4)
    return final_win_prob

# Update the run_simulation function to pass the output widget
def run_simulation(b):
    with output_area:
        output_area.clear_output() # Clear previous output
        print("Starting simulation...")

        active_tab_index = tab_widget.selected_index
        active_level = tab_widget.get_title(active_tab_index)
        active_textarea = input_widgets[active_level]
        pattern_string = active_textarea.value

        lines = pattern_string.strip().split('\n')
        patterns = []
        for line in lines:
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            if pattern:
                patterns.append(pattern)

        if not patterns:
            output_area.clear_output() # Clear "Starting simulation..." message
            print(f"No valid patterns provided for {active_level} level.")
            return

        num_rounds = 5
        num_simulations = 1000 # Increase simulations slightly for better progress visibility

        # Pass the output_area widget to the simulation function
        win_prob = simulate_markov_game(patterns, rounds=num_rounds, simulations=num_simulations, output=output_area)

        # Display the final results
        output_area.clear_output() # Clear final progress message
        print(f"🎯 Final Win Probability Using Markov-Based Prediction Strategy ({active_level} - {num_rounds} Rounds, {num_simulations} Simulations):")
        print(f"{active_level}: {win_prob * 100:.2f}%")


# Link the function to the button's on_click event
run_button.on_click(run_simulation)

## Explore alternative visualization libraries

### Subtask:
Investigate if libraries like `bqplot` or `plotly` within the Jupyter environment offer better capabilities for real-time plotting or updating visual elements as data is generated.


## Implement real-time visualization

### Subtask:
Based on the refactored simulation and explored libraries, implement the visual update mechanism to display the simulation progress or results in near real-time.


**Reasoning**:
Import the necessary modules from `bqplot` for creating a figure, scales, and a line mark, which will be used to visualize the simulation progress.



In [ ]:
from bqplot import Figure, Lines, LinearScale

**Reasoning**:
Modify the `run_simulation` function to initialize the bqplot visualization elements and data lists before starting the simulation. Also, modify `simulate_markov_game` to update these data lists and the bqplot mark within the loop, and display the plot in the output area.



In [ ]:
def run_simulation(b):
    with output_area:
        output_area.clear_output() # Clear previous output
        print("Setting up visualization and starting simulation...")

        active_tab_index = tab_widget.selected_index
        active_level = tab_widget.get_title(active_tab_index)
        active_textarea = input_widgets[active_level]
        pattern_string = active_textarea.value

        lines = pattern_string.strip().split('\n')
        patterns = []
        for line in lines:
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            if pattern:
                patterns.append(pattern)

        if not patterns:
            output_area.clear_output() # Clear "Setting up visualization..." message
            print(f"No valid patterns provided for {active_level} level.")
            return

        num_rounds = 5
        num_simulations = 1000

        # 3. Set up the initial visualization elements and data lists
        x_data = []
        y_data = []

        x_scale = LinearScale()
        y_scale = LinearScale()

        lines_mark = Lines(x=x_data, y=y_data, scales={'x': x_scale, 'y': y_scale})
        fig = Figure(marks=[lines_mark], title=f'{active_level} Win Probability Over Simulations',
                     animation_duration=200) # Add animation for smoother updates

        # Display the figure in the output area
        display(fig)

        # Pass data lists and the lines mark to the simulation function
        win_prob = simulate_markov_game(patterns, rounds=num_rounds, simulations=num_simulations,
                                        output=output_area, x_data=x_data, y_data=y_data, lines_mark=lines_mark)

        # Display the final results below the plot
        with output_area:
             print(f"\n🎯 Final Win Probability Using Markov-Based Prediction Strategy ({active_level} - {num_rounds} Rounds, {num_simulations} Simulations):")
             print(f"{active_level}: {win_prob * 100:.2f}%")


# Modify simulate_markov_game to accept and update plot data and mark
def simulate_markov_game(false_patterns, rounds=5, simulations=100, output=None, x_data=None, y_data=None, lines_mark=None):
    """
    Simulates the game multiple times using a Markov-based prediction strategy,
    updating the output widget and plot periodically.

    Args:
        false_patterns: A list of lists, where each inner list is a sequence
                        of false directions for a specific difficulty level.
        rounds: The number of rounds to play in each simulated game.
        simulations: The number of games to simulate.
        output: An ipywidgets.Output widget to display real-time progress.
        x_data: List to store simulation numbers for plotting.
        y_data: List to store current win probabilities for plotting.
        lines_mark: The bqplot Lines mark to update with new data.


    Returns:
        The final win probability (as a float between 0 and 1) over all simulations.
    """
    flat_false = [item for sublist in false_patterns for item in sublist]
    if not flat_false: # Handle case with no false patterns
        if output:
            with output:
                print("No false patterns provided. Win probability is 0.00%.")
        return 0.0

    transitions = extract_transitions(false_patterns)
    model = build_markov_model(transitions)

    win_count = 0

    for i in range(simulations):
        success = True
        last_false = random.choice(directions)

        for _ in range(rounds):
            if not flat_false:
                 actual_false = random.choice(directions)
            else:
                actual_false = random.choice(flat_false)

            predicted_false = predict_next_false(model, last_false)
            safe_choices = [d for d in directions if d != predicted_false]

            if not safe_choices:
                 success = False
                 break

            possible_correct_answers = [d for d in directions if d != actual_false and d != predicted_false]

            if not possible_correct_answers:
                 success = False
                 break

            correct_answer = random.choice(possible_correct_answers)
            player_choice = random.choice(safe_choices)

            if player_choice != correct_answer:
                success = False
                break

            last_false = actual_false

        if success:
            win_count += 1

        # Update output and plot periodically
        if (i + 1) % (simulations // 50 or 1) == 0 or (i + 1) == simulations: # Update more frequently for visualization
             current_prob = round(win_count / (i + 1), 4)

             # 4. Append data to lists
             if x_data is not None and y_data is not None:
                 x_data.append(i + 1)
                 y_data.append(current_prob)

             # 5. Update the data of the plotting mark
             if lines_mark is not None:
                 # Need to create copies or slices to trigger update in bqplot
                 lines_mark.x = list(x_data)
                 lines_mark.y = list(y_data)

             # 6. Update text output in the output area
             if output:
                 with output:
                    output.clear_output(wait=True)
                    print(f"Simulations complete: {i + 1} / {simulations}")
                    print(f"Current Win Probability: {current_prob * 100:.2f}%")


    final_win_prob = round(win_count / simulations, 4)
    return final_win_prob

# Link the function to the button's on_click event
run_button.on_click(run_simulation)

## Connect ui to real-time updates

### Subtask:
Ensure the `run_simulation` function correctly triggers the real-time updates and manages the display of the interactive visualization.


## Refine user feedback

### Subtask:
Enhance the UI to provide clear feedback to the user, such as progress bars or dynamic text updates, indicating the simulation's status.


**Reasoning**:
Add a progress bar widget to the UI layout and modify the `simulate_markov_game` function to update the progress bar and textual feedback as the simulation progresses.



In [ ]:
from ipywidgets import IntProgress, HTML

# Add a progress bar and status text widget
simulation_progress = IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    orientation='horizontal',
    layout={'width': 'auto'}
)

simulation_status_text = HTML(value="")

# Update the UI layout to include the progress bar and status text
ui = VBox([tab_widget, run_button, simulation_progress, simulation_status_text, output_area])
display(ui)


# Modify simulate_markov_game to update progress bar and status text
def simulate_markov_game(false_patterns, rounds=5, simulations=100, output=None, x_data=None, y_data=None, lines_mark=None, progress_bar=None, status_text=None):
    """
    Simulates the game multiple times using a Markov-based prediction strategy,
    updating output widgets and plot periodically.

    Args:
        false_patterns: A list of lists, where each inner list is a sequence
                        of false directions for a specific difficulty level.
        rounds: The number of rounds to play in each simulated game.
        simulations: The number of games to simulate.
        output: An ipywidgets.Output widget for text output.
        x_data: List to store simulation numbers for plotting.
        y_data: List to store current win probabilities for plotting.
        lines_mark: The bqplot Lines mark to update with new data.
        progress_bar: An ipywidgets.IntProgress widget to update simulation progress.
        status_text: An ipywidgets.HTML widget for dynamic status text.


    Returns:
        The final win probability (as a float between 0 and 1) over all simulations.
    """
    flat_false = [item for sublist in false_patterns for item in sublist]
    if not flat_false: # Handle case with no false patterns
        if output:
            with output:
                print("No false patterns provided. Win probability is 0.00%.")
        if progress_bar:
            progress_bar.value = 100
        if status_text:
            status_text.value = "Simulation finished (No patterns provided)."
        return 0.0

    transitions = extract_transitions(false_patterns)
    model = build_markov_model(transitions)

    win_count = 0

    for i in range(simulations):
        success = True
        last_false = random.choice(directions)

        for _ in range(rounds):
            if not flat_false:
                 actual_false = random.choice(directions)
            else:
                actual_false = random.choice(flat_false)

            predicted_false = predict_next_false(model, last_false)
            safe_choices = [d for d in directions if d != predicted_false]

            if not safe_choices:
                 success = False
                 break

            possible_correct_answers = [d for d in directions if d != actual_false and d != predicted_false]

            if not possible_correct_answers:
                 success = False
                 break

            correct_answer = random.choice(possible_correct_answers)
            player_choice = random.choice(safe_choices)

            if player_choice != correct_answer:
                success = False
                break

            last_false = actual_false

        if success:
            win_count += 1

        # Update output, plot, progress bar, and status text periodically
        if (i + 1) % (simulations // 100 or 1) == 0 or (i + 1) == simulations: # Update more frequently
             current_prob = round(win_count / (i + 1), 4)

             # Update plot data
             if x_data is not None and y_data is not None:
                 x_data.append(i + 1)
                 y_data.append(current_prob)

             if lines_mark is not None:
                 lines_mark.x = list(x_data)
                 lines_mark.y = list(y_data)

             # Update progress bar
             if progress_bar:
                 progress_bar.value = int(((i + 1) / simulations) * 100)

             # Update status text
             if status_text:
                 status_text.value = f"Simulation {i + 1}/{simulations} complete. Current win rate: {current_prob * 100:.2f}%"


    final_win_prob = round(win_count / simulations, 4)
    return final_win_prob


# Update the run_simulation function to pass progress bar and status text
def run_simulation(b):
    with output_area:
        output_area.clear_output() # Clear previous output
        # No longer need "Starting simulation..." here as status text handles it

        # Reset progress bar and status text
        simulation_progress.value = 0
        simulation_status_text.value = "Initializing simulation..."


        active_tab_index = tab_widget.selected_index
        active_level = tab_widget.get_title(active_tab_index)
        active_textarea = input_widgets[active_level]
        pattern_string = active_textarea.value

        lines = pattern_string.strip().split('\n')
        patterns = []
        for line in lines:
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            if pattern:
                patterns.append(pattern)

        if not patterns:
            output_area.clear_output() # Clear plot setup message if any
            simulation_progress.value = 0
            simulation_status_text.value = f"No valid patterns provided for {active_level} level."
            # Remove plot if it was displayed before error
            # Re-display the UI without the figure if needed, or handle figure clearing
            # For simplicity, just clear the text output and update status text
            return

        num_rounds = 5
        num_simulations = 2000 # Increase simulations for smoother progress bar

        # Set up the initial visualization elements and data lists
        x_data = []
        y_data = []

        x_scale = LinearScale()
        y_scale = LinearScale()

        lines_mark = Lines(x=[], y=[], scales={'x': x_scale, 'y': y_scale}) # Start with empty data for plot
        fig = Figure(marks=[lines_mark], title=f'{active_level} Win Probability Over Simulations',
                     animation_duration=100) # Adjust animation duration

        # Display the figure in the output area
        with output_area: # Display plot within output area context
            display(fig)


        # Pass data lists, the lines mark, progress bar, and status text to the simulation function
        win_prob = simulate_markov_game(patterns, rounds=num_rounds, simulations=num_simulations,
                                        output=output_area, x_data=x_data, y_data=y_data, lines_mark=lines_mark,
                                        progress_bar=simulation_progress, status_text=simulation_status_text)

        # Display the final results below the plot and update status text
        with output_area:
             print(f"\n🎯 Final Win Probability Using Markov-Based Prediction Strategy ({active_level} - {num_rounds} Rounds, {num_simulations} Simulations):")
             print(f"{active_level}: {win_prob * 100:.2f}%")

        if simulation_status_text:
             simulation_status_text.value = "Simulation finished."


# Link the function to the button's on_click event
run_button.on_click(run_simulation)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## Summary:

### Data Analysis Key Findings

*   The `simulate_markov_game` function was successfully refactored to yield or update results within the simulation loop, enabling real-time progress display.
*   The implementation utilizes `ipywidgets.Output` for text updates and `bqplot` for plotting, displaying simulation progress and current win probability periodically.
*   The `bqplot` library effectively handles real-time updates by modifying the `x` and `y` data properties of the `Lines` mark within the simulation loop.
*   The user interface was enhanced with an `ipywidgets.IntProgress` bar and an `ipywidgets.HTML` status text widget, providing clear visual and textual feedback on the simulation's progress.
*   The `run_simulation` function orchestrates the process by setting up the necessary widgets and plot elements, passing them to the simulation function, and managing their display and reset.

### Insights or Next Steps

*   The current implementation updates the plot by replacing the entire data lists (`lines_mark.x = list(x_data)`). For simulations with extremely large numbers of iterations, exploring more memory-efficient ways to update `bqplot` marks incrementally could be beneficial.
*   Consider adding options for users to control the update frequency of the plot and text feedback, allowing a balance between responsiveness and performance for longer simulations.


# Task
Implement a real-time interactive simulation of a game where the user provides "false answers" incrementally. The system should use these inputs to build and update a Markov model in real-time, predicting and displaying probable future "false answer" patterns as the simulation progresses. The UI should include controls to advance the simulation and display the real-time predictions.

## Modify the simulation function

### Subtask:
Adapt the `simulate_markov_game` function (or create a new one) to process the "false answers" one by one or in small batches, simulating real-time input.


**Reasoning**:
Create a new function `simulate_interactive_game` that takes the current state (model, last false answer) and a new false answer, updates the model, and returns the new state. This function will process input incrementally.



In [ ]:
def simulate_interactive_game(current_model, last_false, new_false):
    """
    Simulates a single step in the game, updating the Markov model based on a new false answer.

    Args:
        current_model: The current Markov model (defaultdict(Counter)).
        last_false: The last false direction processed.
        new_false: The new false direction received.

    Returns:
        A tuple containing:
            - The updated Markov model.
            - The new last_false direction.
    """
    # Update the transitions list (implicitly by updating the model)
    # We need to rebuild the model or update counts and re-normalize
    # Rebuilding is simpler for small updates, but less efficient for large models
    # Let's update counts and re-normalize for efficiency

    # Create a copy to avoid modifying the original model passed in if needed elsewhere
    updated_model = defaultdict(lambda: Counter(), {k: Counter(v) for k, v in current_model.items()})

    # Add the new transition
    if last_false is not None:
        updated_model[last_false][new_false] += 1

    # Re-normalize the probabilities for the affected 'from' state
    if last_false is not None and last_false in updated_model:
        total = sum(updated_model[last_false].values())
        if total > 0:
            for to_dir in updated_model[last_false]:
                updated_model[last_false][to_dir] /= total

    # Update the last false direction
    new_last_false = new_false

    return updated_model, new_last_false

# Initial state for the interactive game
# We can start with an empty model or a model built from initial patterns
initial_false_patterns = false_answers['Am'] # Example: use 'Am' level initial patterns
initial_transitions = extract_transitions(initial_false_patterns)
initial_markov_model = build_markov_model(initial_transitions)
initial_last_false = initial_false_patterns[-1][-1] if initial_false_patterns else None # Last direction of the last pattern

# Example usage (for testing the function):
# Assuming a new false answer 'R' is received after the initial patterns
# updated_model, new_last_false = simulate_interactive_game(initial_markov_model, initial_last_false, 'R')
# print("Updated Model:", updated_model)
# print("New Last False:", new_last_false)

## Incrementally update the markov model

### Subtask:
Implement a mechanism to update the Markov model with the new data received during the simulated game. This means we won't rebuild the entire model each time, but rather add the new transitions.


**Reasoning**:
Modify the `simulate_interactive_game` function to directly update the provided `current_model` and re-normalize the probabilities for the affected state, avoiding the creation of a deep copy.



In [ ]:
def simulate_interactive_game(current_model, last_false, new_false):
    """
    Simulates a single step in the game, updating the Markov model in place.

    Args:
        current_model: The current Markov model (defaultdict(Counter)). This model will be updated in place.
        last_false: The last false direction processed.
        new_false: The new false direction received.

    Returns:
        The new last_false direction.
    """
    # Update the counts for the new transition
    if last_false is not None:
        current_model[last_false][new_false] += 1

    # Re-normalize the probabilities for the affected 'from' state
    if last_false is not None and last_false in current_model:
        total = sum(current_model[last_false].values())
        if total > 0:
            for to_dir in list(current_model[last_false].keys()): # Use list() to avoid RuntimeError during iteration and modification
                current_model[last_false][to_dir] /= total
        # If total is 0 (shouldn't happen if last_false was in model and got a new_false), probabilities remain 0.0

    # Update the last false direction
    new_last_false = new_false

    return new_last_false

## Predict and display probable patterns

### Subtask:
After processing new data, use the updated Markov model to predict the most probable next false patterns and display these predictions in the UI. This display should be dynamic, updating as the simulated game progresses.


**Reasoning**:
Define the function to predict the next most probable patterns based on the current Markov model and the last false direction, format the predictions, and include a new HTML widget in the UI to display these predictions.



In [ ]:
from ipywidgets import HTML

# Add a widget to display predicted next patterns
predicted_patterns_html = HTML(value="Predicted next patterns will appear here.")

# Update the UI layout to include the predicted patterns widget
# Inserting it before the output_area for better flow
ui = VBox([tab_widget, run_button, simulation_progress, simulation_status_text, predicted_patterns_html, output_area])
display(ui)


def predict_next_patterns(model, last_false, top_n=3):
    """
    Predicts the most probable next false patterns based on the Markov model.

    Args:
        model: The current Markov model (defaultdict of Counters).
        last_false: The last false direction processed.
        top_n: The number of top predictions to display.

    Returns:
        A formatted HTML string displaying the top predicted next patterns.
    """
    predictions = {}
    if last_false in model:
        # Sort transitions by probability in descending order
        sorted_transitions = sorted(model[last_false].items(), key=lambda item: item[1], reverse=True)
        predictions = dict(sorted_transitions)

    # Format the predictions for display
    html_output = f"<b>Predicted next patterns after '{last_false}':</b><br>"
    if not predictions:
        html_output += "No transition data available for this last direction."
    else:
        count = 0
        for direction, prob in predictions.items():
            if count < top_n:
                html_output += f"&nbsp;&nbsp;- {direction}: {prob * 100:.2f}% chance<br>"
                count += 1
            else:
                break # Stop after getting top_n predictions

    return html_output

# Modify run_simulation to orchestrate the interactive game steps and update predictions
def run_simulation(b):
    with output_area:
        output_area.clear_output() # Clear previous plot/text output

        # Reset progress bar and status text
        simulation_progress.value = 0
        simulation_status_text.value = "Initializing simulation..."
        predicted_patterns_html.value = "Predicted next patterns will appear here." # Reset predictions

        active_tab_index = tab_widget.selected_index
        active_level = tab_widget.get_title(active_tab_index)
        active_textarea = input_widgets[active_level]
        pattern_string = active_textarea.value

        lines = pattern_string.strip().split('\n')
        all_patterns = []
        for line in lines:
            pattern = [d.strip() for d in line.split(',') if d.strip()]
            if pattern:
                all_patterns.extend(pattern) # Flatten all patterns into a single list of directions

        if not all_patterns:
            output_area.clear_output()
            simulation_progress.value = 0
            simulation_status_text.value = f"No valid patterns provided for {active_level} level."
            predicted_patterns_html.value = "Cannot predict without input patterns."
            return

        # Initialize the Markov model with a small subset of initial data or keep it empty initially
        # For this interactive simulation, let's process the initial patterns one by one or in batches
        # A simpler approach for this subtask is to just simulate steps and predict based on accumulated data

        # Let's simulate processing the input patterns step by step
        current_model = defaultdict(lambda: Counter())
        last_false = None # Start with no last direction

        num_simulated_steps = len(all_patterns) # Simulate processing each direction in the input patterns

        # Set up the initial visualization elements and data lists
        # We will plot the cumulative win probability if we were simulating games here.
        # Since we are focusing on incremental model updates and predictions, the plot
        # might not be the best fit for this specific subtask's output.
        # Let's keep the plot setup but understand its relevance might change.

        x_data = []
        y_data = []

        x_scale = LinearScale()
        y_scale = LinearScale()

        # Start with empty data for plot, it will be updated as we simulate steps
        lines_mark = Lines(x=[], y=[], scales={'x': x_scale, 'y': y_scale})
        fig = Figure(marks=[lines_mark], title=f'{active_level} Simulation Progress and Win Probability',
                     animation_duration=100)

        with output_area:
            display(fig)


        win_count_cumulative = 0 # To calculate cumulative win probability (if simulating outcomes)
        total_simulated_games = 0 # Number of implied 'games' or rounds simulated

        for i, new_false in enumerate(all_patterns):
            # Simulate a game step (update model based on new_false)
            # The simulate_interactive_game function updates the model in place
            last_false = simulate_interactive_game(current_model, last_false, new_false)

            # After each step, predict the next probable patterns and update the UI
            predicted_html = predict_next_patterns(current_model, last_false)
            predicted_patterns_html.value = predicted_html

            # Optional: Simulate an outcome for this step to update the plot
            # For simplicity in this subtask, let's just plot a dummy value or
            # the cumulative win rate if we defined how a 'win' is calculated per step.
            # Let's assume a simple win scenario where if player avoids the predicted false, it's a 'win' step.
            # This is a simplification for visualization purposes within this subtask.
            # In a real game, a 'win' is over multiple rounds.

            # Example of a simplified step outcome simulation for plotting:
            predicted_false_for_step = predict_next_false(current_model, last_false) # Use the original predict function for a single prediction
            safe_choices_for_step = [d for d in directions if d != predicted_false_for_step]
            step_successful = False
            if safe_choices_for_step:
                 # Assume player randomly chooses a safe direction
                 player_choice_for_step = random.choice(safe_choices_for_step)
                 # Assume correct answer is not the actual false for this step
                 possible_correct_answers_for_step = [d for d in directions if d != new_false]
                 if possible_correct_answers_for_step:
                      correct_answer_for_step = random.choice(possible_correct_answers_for_step)
                      if player_choice_for_step == correct_answer_for_step:
                           step_successful = True

            if step_successful:
                 win_count_cumulative += 1

            total_simulated_games += 1 # Each step can be considered a mini-round

            # Update plot data and mark
            if x_data is not None and y_data is not None:
                 x_data.append(i + 1) # Step number
                 cumulative_win_prob = win_count_cumulative / total_simulated_games if total_simulated_games > 0 else 0.0
                 y_data.append(cumulative_win_prob)

            if lines_mark is not None:
                 lines_mark.x = list(x_data)
                 lines_mark.y = list(y_data)


            # Update progress bar and status text
            progress_percentage = int(((i + 1) / num_simulated_steps) * 100)
            simulation_progress.value = progress_percentage
            current_win_rate = (win_count_cumulative / total_simulated_games * 100) if total_simulated_games > 0 else 0.0
            simulation_status_text.value = f"Processed step {i + 1}/{num_simulated_steps}. Cumulative win rate: {current_win_rate:.2f}%"

            # Small delay to make the updates visible (optional)
            # import time
            # time.sleep(0.05)


        # After processing all input patterns
        simulation_progress.value = 100
        simulation_status_text.value = "Simulation finished."
        final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0

        # Display final results below the plot and update status text
        with output_area:
             print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
             print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")


# Link the function to the button's on_click event
run_button.on_click(run_simulation)


## Add ui elements for simulated game control

### Subtask:
Include buttons or other widgets in the UI to control the simulated game, such as a "Next Round" button to process the next set of simulated false answers or a button to start/stop the simulation.


**Reasoning**:
Create an HBox to hold the "Run Simulation" and "Next Step" buttons, create an HTML widget to display the current step number, and then update the main UI layout to include these new widgets.



In [ ]:
from ipywidgets import HBox, Button, HTML

# Create the 'Next Step' button
next_step_button = Button(description="Next Step")

# Arrange the buttons horizontally
button_box = HBox([run_button, next_step_button])

# Create a widget to display the current simulation step number
current_step_html = HTML(value="Current Step: 0")

# Update the main UI layout to include the new button box and step number HTML
# Keep the order logical: controls, progress/status, predictions, plot
ui = VBox([tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_html, output_area])
display(ui)

## Update the ui to display predictions

### Subtask:
Modify the UI to include an area where the predicted patterns can be displayed clearly, updating in real-time.


**Reasoning**:
Create a new Output widget specifically for displaying the predicted patterns and update the main UI layout to include this new output widget.



In [ ]:
from ipywidgets import Output

# Create a new Output widget specifically for displaying the predicted patterns
predicted_patterns_output = Output()

# Update the main UI layout to include this new output widget.
# Position it logically within the UI, perhaps below the progress/status indicators and above the simulation plot.
# The current layout is: [tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_html, output_area]
# We will replace predicted_patterns_html with the new predicted_patterns_output
ui = VBox([tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_output, output_area])
display(ui)

**Reasoning**:
Modify the `predict_next_patterns` function to write its output to the new output widget and clear its content at the beginning of the `run_simulation` function and potentially before each new prediction is displayed.



In [ ]:
def predict_next_patterns(model, last_false, output_widget, top_n=3):
    """
    Predicts the most probable next false patterns based on the Markov model
    and displays them in the provided output widget.

    Args:
        model: The current Markov model (defaultdict of Counters).
        last_false: The last false direction processed.
        output_widget: The ipywidgets.Output widget to display the predictions.
        top_n: The number of top predictions to display.
    """
    with output_widget:
        output_widget.clear_output(wait=True) # Clear previous predictions

        predictions = {}
        if last_false in model:
            # Sort transitions by probability in descending order
            sorted_transitions = sorted(model[last_false].items(), key=lambda item: item[1], reverse=True)
            predictions = dict(sorted_transitions)

        print(f"Predicted next patterns after '{last_false}':")
        if not predictions:
            print("No transition data available for this last direction.")
        else:
            count = 0
            for direction, prob in predictions.items():
                if count < top_n:
                    print(f"  - {direction}: {prob * 100:.2f}% chance")
                    count += 1
                else:
                    break # Stop after getting top_n predictions


# Modify run_simulation to orchestrate the interactive game steps and update predictions using the new output widget
def run_simulation(b):
    with output_area:
        output_area.clear_output() # Clear previous plot/text output

    # Reset progress bar and status text
    simulation_progress.value = 0
    simulation_status_text.value = "Initializing simulation..."
    current_step_html.value = "Current Step: 0"

    # Clear the predicted patterns output area
    with predicted_patterns_output:
        predicted_patterns_output.clear_output()


    active_tab_index = tab_widget.selected_index
    active_level = tab_widget.get_title(active_tab_index)
    active_textarea = input_widgets[active_level]
    pattern_string = active_textarea.value

    lines = pattern_string.strip().split('\n')
    all_patterns = []
    for line in lines:
        pattern = [d.strip() for d in line.split(',') if d.strip()]
        if pattern:
            all_patterns.extend(pattern) # Flatten all patterns into a single list of directions

    if not all_patterns:
        with output_area:
            output_area.clear_output()
        simulation_progress.value = 0
        simulation_status_text.value = f"No valid patterns provided for {active_level} level."
        with predicted_patterns_output:
             predicted_patterns_output.clear_output()
             print("Cannot predict without input patterns.")
        return

    # Initialize the Markov model
    current_model = defaultdict(lambda: Counter())
    last_false = None # Start with no last direction

    num_simulated_steps = len(all_patterns) # Simulate processing each direction in the input patterns

    # Set up the initial visualization elements and data lists
    x_data = []
    y_data = []

    x_scale = LinearScale()
    y_scale = LinearScale()

    lines_mark = Lines(x=[], y=[], scales={'x': x_scale, 'y': y_scale}) # Start with empty data for plot
    fig = Figure(marks=[lines_mark], title=f'{active_level} Simulation Progress and Win Probability',
                 animation_duration=100)

    with output_area: # Display the figure in the output area
        display(fig)


    win_count_cumulative = 0 # To calculate cumulative win probability
    total_simulated_games = 0 # Number of implied 'games' or rounds simulated

    # --- Interactive Simulation Loop (now driven by a 'Next Step' button later) ---
    # For now, we will process all steps in the input data when 'Run Simulation' is clicked
    # This will be refactored in a later subtask to use the 'Next Step' button.

    for i, new_false in enumerate(all_patterns):
        # Simulate a game step (update model based on new_false)
        last_false = simulate_interactive_game(current_model, last_false, new_false)

        # Update the current step number display
        current_step_html.value = f"Current Step: {i + 1}"


        # After each step, predict the next probable patterns and update the dedicated output widget
        predict_next_patterns(current_model, last_false, predicted_patterns_output)


        # Example of a simplified step outcome simulation for plotting:
        predicted_false_for_step = predict_next_false(current_model, last_false)
        safe_choices_for_step = [d for d in directions if d != predicted_false_for_step]
        step_successful = False
        if safe_choices_for_step:
             player_choice_for_step = random.choice(safe_choices_for_step)
             possible_correct_answers_for_step = [d for d in directions if d != new_false]
             if possible_correct_answers_for_step:
                  correct_answer_for_step = random.choice(possible_correct_answers_for_step)
                  if player_choice_for_step == correct_answer_for_step:
                       step_successful = True

        if step_successful:
             win_count_cumulative += 1

        total_simulated_games += 1 # Each step can be considered a mini-round

        # Update plot data and mark
        if x_data is not None and y_data is not None:
             x_data.append(i + 1) # Step number
             cumulative_win_prob = win_count_cumulative / total_simulated_games if total_simulated_games > 0 else 0.0
             y_data.append(cumulative_win_prob)

        if lines_mark is not None:
             lines_mark.x = list(x_data)
             lines_mark.y = list(y_data)


        # Update progress bar and status text
        progress_percentage = int(((i + 1) / num_simulated_steps) * 100)
        simulation_progress.value = progress_percentage
        current_win_rate = (win_count_cumulative / total_simulated_games * 100) if total_simulated_games > 0 else 0.0
        simulation_status_text.value = f"Processed step {i + 1}/{num_simulated_steps}. Cumulative win rate: {current_win_rate:.2f}%"


    # After processing all input patterns
    simulation_progress.value = 100
    simulation_status_text.value = "Simulation finished."


    # Display final cumulative win probability below the plot
    with output_area:
         print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
         final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0
         print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")


# Link the 'Run Simulation' function to the button's on_click event
run_button.on_click(run_simulation)

# The 'Next Step' button functionality will be implemented in a future subtask
# def on_next_step_button_clicked(b):
#     # Logic to process one step and update UI
#     pass
# next_step_button.on_click(on_next_step_button_clicked)

## Summary:

## Summary of Solving Process

The task involved implementing a real-time interactive simulation of a game where user-provided "false answers" incrementally update a Markov model. This model is then used to predict and display probable future "false answer" patterns in real-time within a user interface that includes controls to advance the simulation.

The process was broken down into several subtasks:

1.  **Modify the simulation function:** The `simulate_interactive_game` function was created/adapted to process "false answers" one by one, simulating real-time input and updating the Markov model incrementally.
2.  **Incrementally update the markov model:** The `simulate_interactive_game` function was further refined to update the Markov model in place by incrementing transition counts and re-normalizing probabilities for the affected state after each new false answer input.
3.  **Predict and display probable patterns:** A function (`predict_next_patterns`) was implemented to use the updated Markov model to predict the most probable next false patterns based on the last processed false answer. An HTML widget was added to the UI to display these predictions, and the `run_simulation` function was modified to call the prediction function and update the UI after each simulated step.
4.  **Add ui elements for simulated game control:** UI elements, specifically a "Next Step" button and an HTML display for the current step number, were added to provide controls for the interactive simulation.
5.  **Update the ui to display predictions:** A dedicated `ipywidgets.Output` widget was created to display the predicted patterns, replacing the previous HTML widget. The `predict_next_patterns` function was modified to write directly to this output widget, ensuring clear and dynamic real-time display of predictions within the UI.

## Data Analysis Key Findings

*   The `simulate_interactive_game` function successfully updates the Markov model incrementally with each new "false answer" input by updating transition counts and re-normalizing probabilities for the affected state.
*   The `predict_next_patterns` function can generate the most probable next "false answer" directions and their associated probabilities based on the current state of the Markov model and the last processed false answer.
*   The user interface now includes dedicated areas for displaying the current simulation step number and the real-time predicted next patterns, which are updated dynamically as the simulation progresses.

## Insights or Next Steps

*   The current implementation processes all input patterns sequentially within the `run_simulation` function. The "Next Step" button added to the UI is not yet functional. The next step should be to refactor the simulation logic to be driven by the "Next Step" button, allowing the user to advance the simulation step by step and observe the real-time model updates and predictions.
*   Consider adding a visual representation of the Markov model or the prediction probabilities (e.g., a bar chart) in the UI to provide a more intuitive understanding of the model's current state and predictions.


# Task
Implement an interactive simulation with step-by-step control and probability visualization based on the provided Python code.

## Prepare for step-by-step simulation

### Subtask:
Modify the `run_simulation` function to initialize the necessary state (Markov model, last false answer, data lists, plot, etc.) and store it in a way that can be accessed by the "Next Step" button's callback function. Do not run the full simulation loop in `run_simulation`.


**Reasoning**:
Modify the `run_simulation` function to initialize the state for the interactive game and store it globally or in a container, remove the simulation loop from this function, and enable the 'Next Step' button. I will use a dictionary to store the state.



In [ ]:
# Dictionary to hold the state of the interactive simulation
interactive_state = {}

def run_simulation(b):
    global interactive_state # Declare as global to store state

    with output_area:
        output_area.clear_output() # Clear previous plot/text output

    # Reset progress bar and status text
    simulation_progress.value = 0
    simulation_status_text.value = "Initializing simulation..."
    current_step_html.value = "Current Step: 0"

    # Clear the predicted patterns output area
    with predicted_patterns_output:
        predicted_patterns_output.clear_output()

    active_tab_index = tab_widget.selected_index
    active_level = tab_widget.get_title(active_tab_index)
    active_textarea = input_widgets[active_level]
    pattern_string = active_textarea.value

    lines = pattern_string.strip().split('\n')
    all_patterns = []
    for line in lines:
        pattern = [d.strip() for d in line.split(',') if d.strip()]
        if pattern:
            all_patterns.extend(pattern) # Flatten all patterns into a single list of directions

    if not all_patterns:
        with output_area:
            output_area.clear_output()
        simulation_progress.value = 0
        simulation_status_text.value = f"No valid patterns provided for {active_level} level."
        with predicted_patterns_output:
             predicted_patterns_output.clear_output()
             print("Cannot predict without input patterns.")
        next_step_button.disabled = True # Disable next step if no patterns
        return

    # Initialize the Markov model
    current_model = defaultdict(lambda: Counter())
    last_false = None # Start with no last direction

    num_simulated_steps = len(all_patterns) # Total steps to simulate

    # Set up the initial visualization elements and data lists
    x_data = []
    y_data = []

    x_scale = LinearScale()
    y_scale = LinearScale()

    lines_mark = Lines(x=[], y=[], scales={'x': x_scale, 'y': y_scale}) # Start with empty data for plot
    fig = Figure(marks=[lines_mark], title=f'{active_level} Simulation Progress and Cumulative Win Probability',
                 animation_duration=100)

    with output_area: # Display the figure in the output area
        display(fig)

    # Initialize cumulative win tracking
    win_count_cumulative = 0
    total_simulated_games = 0 # Each step is a mini-round for cumulative win calc


    # Store the state in the global dictionary
    interactive_state = {
        'current_model': current_model,
        'last_false': last_false,
        'all_patterns': all_patterns,
        'x_data': x_data,
        'y_data': y_data,
        'lines_mark': lines_mark,
        'fig': fig, # Store figure reference if needed later
        'num_simulated_steps': num_simulated_steps,
        'win_count_cumulative': win_count_cumulative,
        'total_simulated_games': total_simulated_games,
        'current_step_index': 0, # Track which pattern we are processing
        'current_step_html': current_step_html,
        'simulation_progress': simulation_progress,
        'simulation_status_text': simulation_status_text,
        'predicted_patterns_output': predicted_patterns_output,
        'output_area': output_area, # Store output area reference
        'active_level': active_level # Store active level for labels
    }

    # Enable the 'Next Step' button
    next_step_button.disabled = False

    simulation_status_text.value = "Simulation initialized. Ready to process steps."
    predict_next_patterns(interactive_state['current_model'], interactive_state['last_false'], interactive_state['predicted_patterns_output'])


# Link the 'Run Simulation' function to the button's on_click event
run_button.on_click(run_simulation)

# Disable the 'Next Step' button initially
next_step_button.disabled = True

# The 'Next Step' button functionality will be implemented in the next subtask
# def on_next_step_button_clicked(b):
#     # Logic to process one step using interactive_state
#     pass
# next_step_button.on_click(on_next_step_button_clicked)


## Implement the "next step" button logic

### Subtask:
Create a callback function for the "Next Step" button. This function should retrieve the current simulation state, process the next false answer from the input patterns using the `simulate_interactive_game` function, update the Markov model, update the plot data and the bqplot lines mark, update the step number, progress bar, and status text, predict and display the next probable patterns using the `predict_next_patterns` function, and update the stored simulation state.


**Reasoning**:
Define the `on_next_step_button_clicked` function to retrieve the current simulation state, process the next false answer, update the model and plot data, update the UI widgets, predict and display patterns, update the step index, and store the updated state back.



In [ ]:
def on_next_step_button_clicked(b):
    global interactive_state # Access the global state

    # Check if the simulation has been initialized
    if not interactive_state:
        with interactive_state.get('output_area', output_area):
            print("Please run the simulation first.")
        return

    # Retrieve the current state
    current_model = interactive_state['current_model']
    last_false = interactive_state['last_false']
    all_patterns = interactive_state['all_patterns']
    x_data = interactive_state['x_data']
    y_data = interactive_state['y_data']
    lines_mark = interactive_state['lines_mark']
    num_simulated_steps = interactive_state['num_simulated_steps']
    win_count_cumulative = interactive_state['win_count_cumulative']
    total_simulated_games = interactive_state['total_simulated_games']
    current_step_index = interactive_state['current_step_index']
    current_step_html = interactive_state['current_step_html']
    simulation_progress = interactive_state['simulation_progress']
    simulation_status_text = interactive_state['simulation_status_text']
    predicted_patterns_output = interactive_state['predicted_patterns_output']
    output_area = interactive_state['output_area']
    active_level = interactive_state['active_level']


    # Check if there are more steps to process
    if current_step_index >= num_simulated_steps:
        simulation_status_text.value = "Simulation finished."
        next_step_button.disabled = True # Disable the button when done
        return

    # 3. Get the next false answer
    new_false = all_patterns[current_step_index]

    # 4. Call simulate_interactive_game to update the Markov model
    # This function updates the model in place and returns the new last_false
    new_last_false = simulate_interactive_game(current_model, last_false, new_false)

    # Update the last_false in the state
    interactive_state['last_false'] = new_last_false

    # 5. Update the plot data lists and the bqplot lines mark
    # Example of a simplified step outcome simulation for plotting:
    predicted_false_for_step = predict_next_false(current_model, new_last_false) # Use the original predict function
    safe_choices_for_step = [d for d in directions if d != predicted_false_for_step]
    step_successful = False
    if safe_choices_for_step:
         player_choice_for_step = random.choice(safe_choices_for_step)
         possible_correct_answers_for_step = [d for d in directions if d != new_false]
         if possible_correct_answers_for_step:
              correct_answer_for_step = random.choice(possible_correct_answers_for_step)
              if player_choice_for_step == correct_answer_for_step:
                   step_successful = True

    if step_successful:
         win_count_cumulative += 1

    total_simulated_games += 1 # Each step is a mini-round for cumulative win calc

    # Update the state with new cumulative win data
    interactive_state['win_count_cumulative'] = win_count_cumulative
    interactive_state['total_simulated_games'] = total_simulated_games


    # Append data to lists
    x_data.append(current_step_index + 1) # Step number
    cumulative_win_prob = win_count_cumulative / total_simulated_games if total_simulated_games > 0 else 0.0
    y_data.append(cumulative_win_prob)

    # Update the data of the plotting mark
    if lines_mark is not None:
        # Need to create copies or slices to trigger update in bqplot
        lines_mark.x = list(x_data)
        lines_mark.y = list(y_data)


    # 6. Update the UI widgets
    current_step_html.value = f"Current Step: {current_step_index + 1}"

    progress_percentage = int(((current_step_index + 1) / num_simulated_steps) * 100)
    simulation_progress.value = progress_percentage

    current_win_rate = (win_count_cumulative / total_simulated_games * 100) if total_simulated_games > 0 else 0.0
    simulation_status_text.value = f"Processed step {current_step_index + 1}/{num_simulated_steps}. Cumulative win rate: {current_win_rate:.2f}%"

    # 7. Call predict_next_patterns to display the updated predictions
    predict_next_patterns(current_model, new_last_false, predicted_patterns_output)

    # 8. Increment current_step_index and update the state
    interactive_state['current_step_index'] = current_step_index + 1

    # If this was the last step, display final results
    if interactive_state['current_step_index'] >= num_simulated_steps:
        simulation_status_text.value = "Simulation finished."
        next_step_button.disabled = True # Disable the button when done
        final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0
        with output_area:
             print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
             print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")


# 9. Link this function to the next_step_button.on_click event
next_step_button.on_click(on_next_step_button_clicked)

## Add a bar chart for prediction probabilities

### Subtask:
Add a bar chart for prediction probabilities.


**Reasoning**:
Import the necessary modules from `bqplot` for creating scales (`OrdinalScale`, `LinearScale`) and a bar mark (`Bars`), then update the UI layout to include the new bar chart figure.



In [43]:
from bqplot import OrdinalScale, LinearScale, Bars, Figure

# 2. Create new scales for the x-axis (categorical, using OrdinalScale) and y-axis (linear, using LinearScale) for the bar chart.
x_scale_bar = OrdinalScale()
y_scale_bar = LinearScale()

# 3. Create a Bars mark, assigning the new scales to its x_scale and y_scale attributes.
# Initialize the x and y data of the bar mark with empty lists or default values.
bar_mark = Bars(x=[], y=[], scales={'x': x_scale_bar, 'y': y_scale_bar}, padding=0.2)

# 4. Create a new Figure object for the bar chart, adding the Bars mark to its marks list. Set a title for the bar chart.
bar_chart_fig = Figure(marks=[bar_mark], title='Next Direction Prediction Probabilities',
                       x_ figyelabel='Direction', y_label='Probability')

# 5. Update the main UI layout (ui = VBox[...]) to include the new bar chart figure.
# Position it logically within the UI, for example, below the predicted patterns text output.
# The current layout is: [tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_output, output_area]
# We will insert the bar_chart_fig after predicted_patterns_output
ui = VBox([tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_output, bar_chart_fig, output_area])
display(ui)

# Now, update the predict_next_patterns and on_next_step_button_clicked functions
# to update the data of this bar chart. This will be done in subsequent steps.

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-43-3313856051.py, line 13)

**Reasoning**:
The previous command failed due to a typo in the `Figure` constructor (`x_ figyelabel` instead of `x_label`). Correct the typo and regenerate the code block to create and display the bar chart and update the UI layout.



In [ ]:
from bqplot import OrdinalScale, LinearScale, Bars, Figure

# 2. Create new scales for the x-axis (categorical, using OrdinalScale) and y-axis (linear, using LinearScale) for the bar chart.
x_scale_bar = OrdinalScale()
y_scale_bar = LinearScale()

# 3. Create a Bars mark, assigning the new scales to its x_scale and y_scale attributes.
# Initialize the x and y data of the bar mark with empty lists or default values.
bar_mark = Bars(x=[], y=[], scales={'x': x_scale_bar, 'y': y_scale_bar}, padding=0.2)

# 4. Create a new Figure object for the bar chart, adding the Bars mark to its marks list. Set a title for the bar chart.
bar_chart_fig = Figure(marks=[bar_mark], title='Next Direction Prediction Probabilities',
                       x_label='Direction', y_label='Probability') # Corrected typo here

# 5. Update the main UI layout (ui = VBox[...]) to include the new bar chart figure.
# Position it logically within the UI, for example, below the predicted patterns text output.
# The current layout is: [tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_output, output_area]
# We will insert the bar_chart_fig after predicted_patterns_output
ui = VBox([tab_widget, button_box, simulation_progress, simulation_status_text, current_step_html, predicted_patterns_output, bar_chart_fig, output_area])
display(ui)

# Now, update the predict_next_patterns and on_next_step_button_clicked functions
# to update the data of this bar chart. This will be done in subsequent steps.

**Reasoning**:
Modify the `predict_next_patterns` function to also update the data of the bar chart using the calculated probabilities and update the `on_next_step_button_clicked` function to pass the bar chart mark to the prediction function.



In [38]:
def predict_next_patterns(model, last_false, output_widget, bar_mark_widget, top_n=3):
    """
    Predicts the most probable next false patterns based on the Markov model
    and displays them in the provided output widget and updates the bar chart.

    Args:
        model: The current Markov model (defaultdict of Counters).
        last_false: The last false direction processed.
        output_widget: The ipywidgets.Output widget to display the predictions.
        bar_mark_widget: The bqplot Bars mark to update with prediction probabilities.
        top_n: The number of top predictions to display.
    """
    predictions = {}
    if last_false in model:
        # Sort transitions by probability in descending order
        sorted_transitions = sorted(model[last_false].items(), key=lambda item: item[1], reverse=True)
        predictions = dict(sorted_transitions)

    # Update the text output widget
    with output_widget:
        output_widget.clear_output(wait=True) # Clear previous predictions

        print(f"Predicted next patterns after '{last_false}':")
        if not predictions:
            print("No transition data available for this last direction.")
        else:
            count = 0
            for direction, prob in predictions.items():
                if count < top_n:
                    print(f"  - {direction}: {prob * 100:.2f}% chance")
                    count += 1
                else:
                    break # Stop after getting top_n predictions

    # Update the bar chart data
    if bar_mark_widget is not None:
        if predictions:
            # Get directions and probabilities for the bar chart
            bar_x_data = list(predictions.keys())
            bar_y_data = list(predictions.values())
            bar_mark_widget.x = bar_x_data
            bar_mark_widget.y = bar_y_data
        else:
            # Clear the bar chart if no predictions are available
            bar_mark_widget.x = []
            bar_mark_widget.y = []


def on_next_step_button_clicked(b):
    global interactive_state # Access the global state

    # Check if the simulation has been initialized
    if not interactive_state:
        with interactive_state.get('output_area', output_area):
            print("Please run the simulation first.")
        return

    # Retrieve the current state
    current_model = interactive_state['current_model']
    last_false = interactive_state['last_false']
    all_patterns = interactive_state['all_patterns']
    x_data = interactive_state['x_data'] # For cumulative win plot
    y_data = interactive_state['y_data'] # For cumulative win plot
    lines_mark = interactive_state['lines_mark'] # For cumulative win plot
    num_simulated_steps = interactive_state['num_simulated_steps']
    win_count_cumulative = interactive_state['win_count_cumulative']
    total_simulated_games = interactive_state['total_simulated_games']
    current_step_index = interactive_state['current_step_index']
    current_step_html = interactive_state['current_step_html']
    simulation_progress = interactive_state['simulation_progress']
    simulation_status_text = interactive_state['simulation_status_text']
    predicted_patterns_output = interactive_state['predicted_patterns_output']
    output_area = interactive_state['output_area']
    active_level = interactive_state['active_level']
    bar_mark_widget = interactive_state.get('bar_mark_widget') # Retrieve the bar mark widget


    # Check if there are more steps to process
    if current_step_index >= num_simulated_steps:
        simulation_status_text.value = "Simulation finished."
        next_step_button.disabled = True # Disable the button when done
        final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0
        with output_area:
             print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
             print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")
        # Clear the prediction bar chart when simulation finishes
        if bar_mark_widget is not None:
             bar_mark_widget.x = []
             bar_mark_widget.y = []
        return

    # 3. Get the next false answer
    new_false = all_patterns[current_step_index]

    # 4. Call simulate_interactive_game to update the Markov model
    # This function updates the model in place and returns the new last_false
    new_last_false = simulate_interactive_game(current_model, last_false, new_false)

    # Update the last_false in the state
    interactive_state['last_false'] = new_last_false

    # 5. Update the plot data lists and the bqplot lines mark (Cumulative Win Plot)
    # Example of a simplified step outcome simulation for plotting:
    predicted_false_for_step = predict_next_false(current_model, new_last_false) # Use the original predict function
    safe_choices_for_step = [d for d in directions if d != predicted_false_for_step]
    step_successful = False
    if safe_choices_for_step:
         player_choice_for_step = random.choice(safe_choices_for_step)
         possible_correct_answers_for_step = [d for d in directions if d != new_false]
         if possible_correct_answers_for_step:
              correct_answer_for_step = random.choice(possible_correct_answers_for_step)
              if player_choice_for_step == correct_answer_for_step:
                   step_successful = True

    if step_successful:
         win_count_cumulative += 1

    total_simulated_games += 1 # Each step is a mini-round for cumulative win calc

    # Update the state with new cumulative win data
    interactive_state['win_count_cumulative'] = win_count_cumulative
    interactive_state['total_simulated_games'] = total_simulated_games


    # Append data to lists
    x_data.append(current_step_index + 1) # Step number
    cumulative_win_prob = win_count_cumulative / total_simulated_games if total_simulated_games > 0 else 0.0
    y_data.append(cumulative_win_prob)

    # Update the data of the plotting mark
    if lines_mark is not None:
        # Need to create copies or slices to trigger update in bqplot
        lines_mark.x = list(x_data)
        lines_mark.y = list(y_data)


    # 6. Update the UI widgets
    current_step_html.value = f"Current Step: {current_step_index + 1}"

    progress_percentage = int(((current_step_index + 1) / num_simulated_steps) * 100)
    simulation_progress.value = progress_percentage

    current_win_rate = (win_count_cumulative / total_simulated_games * 100) if total_simulated_games > 0 else 0.0
    simulation_status_text.value = f"Processed step {current_step_index + 1}/{num_simulated_steps}. Cumulative win rate: {current_win_rate:.2f}%"

    # 7. Call predict_next_patterns to display the updated predictions and update the bar chart
    predict_next_patterns(current_model, new_last_false, predicted_patterns_output, bar_mark_widget)

    # 8. Increment current_step_index and update the state
    interactive_state['current_step_index'] = current_step_index + 1

    # If this was the last step, display final results (handled at the beginning of the function now)
    # if interactive_state['current_step_index'] >= num_simulated_steps:
    #     simulation_status_text.value = "Simulation finished."
    #     next_step_button.disabled = True # Disable the button when done
    #     final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0
    #     with output_area:
    #          print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
    #          print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")
    #     # Clear the prediction bar chart when simulation finishes
    #     if bar_mark_widget is not None:
    #          bar_mark_widget.x = []
    #          bar_mark_widget.y = []


# Link this function to the next_step_button.on_click event
next_step_button.on_click(on_next_step_button_clicked)


# Update run_simulation to store the bar_mark_widget in the state
def run_simulation(b):
    global interactive_state # Declare as global to store state

    with output_area:
        output_area.clear_output() # Clear previous plot/text output

    # Reset progress bar and status text
    simulation_progress.value = 0
    simulation_status_text.value = "Initializing simulation..."
    current_step_html.value = "Current Step: 0"

    # Clear the predicted patterns output area
    with predicted_patterns_output:
        predicted_patterns_output.clear_output()

    # Clear the bar chart
    bar_mark.x = []
    bar_mark.y = []


    active_tab_index = tab_widget.selected_index
    active_level = tab_widget.get_title(active_tab_index)
    active_textarea = input_widgets[active_level]
    pattern_string = active_textarea.value

    lines = pattern_string.strip().split('\n')
    all_patterns = []
    for line in lines:
        pattern = [d.strip() for d in line.split(',') if d.strip()]
        if pattern:
            all_patterns.extend(pattern) # Flatten all patterns into a single list of directions

    if not all_patterns:
        with output_area:
            output_area.clear_output()
        simulation_progress.value = 0
        simulation_status_text.value = f"No valid patterns provided for {active_level} level."
        with predicted_patterns_output:
             predicted_patterns_output.clear_output()
             print("Cannot predict without input patterns.")
        next_step_button.disabled = True # Disable next step if no patterns
        return

    # Initialize the Markov model
    current_model = defaultdict(lambda: Counter())
    last_false = None # Start with no last direction

    num_simulated_steps = len(all_patterns) # Total steps to simulate

    # Set up the initial visualization elements and data lists (Cumulative Win Plot)
    x_data = []
    y_data = []

    x_scale = LinearScale()
    y_scale = LinearScale()

    lines_mark = Lines(x=[], y=[], scales={'x': x_scale, 'y': y_scale}) # Start with empty data for plot
    fig = Figure(marks=[lines_mark], title=f'{active_level} Simulation Progress and Cumulative Win Probability',
                 animation_duration=100)

    with output_area: # Display the figure in the output area
        display(fig)

    # Initialize cumulative win tracking
    win_count_cumulative = 0
    total_simulated_games = 0 # Each step is a mini-round for cumulative win calc


    # Store the state in the global dictionary
    interactive_state = {
        'current_model': current_model,
        'last_false': last_false,
        'all_patterns': all_patterns,
        'x_data': x_data,
        'y_data': y_data,
        'lines_mark': lines_mark,
        'fig': fig, # Store figure reference if needed later
        'num_simulated_steps': num_simulated_steps,
        'win_count_cumulative': win_count_cumulative,
        'total_simulated_games': total_simulated_games,
        'current_step_index': 0, # Track which pattern we are processing
        'current_step_html': current_step_html,
        'simulation_progress': simulation_progress,
        'simulation_status_text': simulation_status_text,
        'predicted_patterns_output': predicted_patterns_output,
        'output_area': output_area, # Store output area reference
        'active_level': active_level, # Store active level for labels
        'bar_mark_widget': bar_mark # Store the bar mark widget
    }

    # Enable the 'Next Step' button
    next_step_button.disabled = False

    simulation_status_text.value = "Simulation initialized. Ready to process steps."
    # Display initial predictions (before processing any steps, based on initial state if any)
    predict_next_patterns(interactive_state['current_model'], interactive_state['last_false'], interactive_state['predicted_patterns_output'], interactive_state['bar_mark_widget'])


# Link the 'Run Simulation' function to the button's on_click event
run_button.on_click(run_simulation)

# Disable the 'Next Step' button initially
next_step_button.disabled = True

## Handle end of simulation

### Subtask:
Add logic to the "Next Step" button's callback to detect when all input patterns have been processed. At the end of the simulation, disable the "Next Step" button, display a final message, and update the UI elements accordingly.


**Reasoning**:
Implement the end-of-simulation logic in the `on_next_step_button_clicked` function, including checking for the end, disabling the button, displaying a final message and cumulative win probability, and clearing the prediction bar chart.



In [39]:
def on_next_step_button_clicked(b):
    global interactive_state # Access the global state

    # Check if the simulation has been initialized
    if not interactive_state:
        with interactive_state.get('output_area', output_area):
            print("Please run the simulation first.")
        return

    # Retrieve the current state
    current_model = interactive_state['current_model']
    last_false = interactive_state['last_false']
    all_patterns = interactive_state['all_patterns']
    x_data = interactive_state['x_data'] # For cumulative win plot
    y_data = interactive_state['y_data'] # For cumulative win plot
    lines_mark = interactive_state['lines_mark'] # For cumulative win plot
    num_simulated_steps = interactive_state['num_simulated_steps']
    win_count_cumulative = interactive_state['win_count_cumulative']
    total_simulated_games = interactive_state['total_simulated_games']
    current_step_index = interactive_state['current_step_index']
    current_step_html = interactive_state['current_step_html']
    simulation_progress = interactive_state['simulation_progress']
    simulation_status_text = interactive_state['simulation_status_text']
    predicted_patterns_output = interactive_state['predicted_patterns_output']
    output_area = interactive_state['output_area']
    active_level = interactive_state['active_level']
    bar_mark_widget = interactive_state.get('bar_mark_widget') # Retrieve the bar mark widget

    # 1. Check if all input patterns have been processed
    if current_step_index >= num_simulated_steps:
        # 2. This indicates the end of the simulation. Set the simulation_status_text.value to a final message
        simulation_status_text.value = "Simulation finished."

        # 3. Disable the "Next Step" button
        next_step_button.disabled = True

        # 4. Display the final cumulative win probability
        final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0
        with output_area:
             print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
             print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")

        # 5. Clear the prediction bar chart
        if bar_mark_widget is not None:
             bar_mark_widget.x = []
             bar_mark_widget.y = []

        # 6. Add a return statement to exit the function
        return

    # 3. Get the next false answer
    new_false = all_patterns[current_step_index]

    # 4. Call simulate_interactive_game to update the Markov model
    # This function updates the model in place and returns the new last_false
    new_last_false = simulate_interactive_game(current_model, last_false, new_false)

    # Update the last_false in the state
    interactive_state['last_false'] = new_last_false

    # 5. Update the plot data lists and the bqplot lines mark (Cumulative Win Plot)
    # Example of a simplified step outcome simulation for plotting:
    predicted_false_for_step = predict_next_false(current_model, new_last_false) # Use the original predict function
    safe_choices_for_step = [d for d in directions if d != predicted_false_for_step]
    step_successful = False
    if safe_choices_for_step:
         player_choice_for_step = random.choice(safe_choices_for_step)
         possible_correct_answers_for_step = [d for d in directions if d != new_false]
         if possible_correct_answers_for_step:
              correct_answer_for_step = random.choice(possible_correct_answers_for_step)
              if player_choice_for_step == correct_answer_for_step:
                   step_successful = True

    if step_successful:
         win_count_cumulative += 1

    total_simulated_games += 1 # Each step is a mini-round for cumulative win calc

    # Update the state with new cumulative win data
    interactive_state['win_count_cumulative'] = win_count_cumulative
    interactive_state['total_simulated_games'] = total_simulated_games


    # Append data to lists
    x_data.append(current_step_index + 1) # Step number
    cumulative_win_prob = win_count_cumulative / total_simulated_games if total_simulated_games > 0 else 0.0
    y_data.append(cumulative_win_prob)

    # Update the data of the plotting mark
    if lines_mark is not None:
        # Need to create copies or slices to trigger update in bqplot
        lines_mark.x = list(x_data)
        lines_mark.y = list(y_data)


    # 6. Update the UI widgets
    current_step_html.value = f"Current Step: {current_step_index + 1}"

    progress_percentage = int(((current_step_index + 1) / num_simulated_steps) * 100)
    simulation_progress.value = progress_percentage

    current_win_rate = (win_count_cumulative / total_simulated_games * 100) if total_simulated_games > 0 else 0.0
    simulation_status_text.value = f"Processed step {current_step_index + 1}/{num_simulated_steps}. Cumulative win rate: {current_win_rate:.2f}%"

    # 7. Call predict_next_patterns to display the updated predictions and update the bar chart
    predict_next_patterns(current_model, new_last_false, predicted_patterns_output, bar_mark_widget)

    # 8. Increment current_step_index and update the state
    interactive_state['current_step_index'] = current_step_index + 1

    # If this was the last step, display final results (handled at the beginning of the function now)
    # if interactive_state['current_step_index'] >= num_simulated_steps:
    #     simulation_status_text.value = "Simulation finished."
    #     next_step_button.disabled = True # Disable the button when done
    #     final_cumulative_win_prob = (win_count_cumulative / total_simulated_games) if total_simulated_games > 0 else 0.0
    #     with output_area:
    #          print(f"\n🎯 Cumulative 'Win' Probability After Processing Input Patterns ({active_level}):")
    #          print(f"Cumulative Rate: {final_cumulative_win_prob * 100:.2f}%")
    #     # Clear the prediction bar chart when simulation finishes
    #     if bar_mark_widget is not None:
    #          bar_mark_widget.x = []
    #          bar_mark_widget.y = []


# Link this function to the next_step_button.on_click event
next_step_button.on_click(on_next_step_button_clicked)

## Summary:

### Data Analysis Key Findings

*   The `run_simulation` function was successfully modified to initialize the interactive simulation state, including the Markov model, data lists for plotting, BQPlot elements, and UI widgets, storing them in a global `interactive_state` dictionary. The simulation loop was removed, and the "Next Step" button is enabled after initialization.
*   A callback function `on_next_step_button_clicked` was created and linked to the "Next Step" button. This function retrieves the simulation state, processes the next input pattern, updates the Markov model and cumulative win probability, appends data to the plot lists, updates the BQPlot lines mark, and refreshes the UI widgets (step number, progress bar, status text).
*   A BQPlot bar chart was added to the UI layout to visualize the prediction probabilities of the next direction based on the updated Markov model.
*   The `predict_next_patterns` function was modified to update the data of the new bar chart with the predicted directions and their probabilities.
*   The `on_next_step_button_clicked` and `run_simulation` functions were updated to manage and pass the bar chart's BQPlot mark, ensuring it's updated dynamically during the step-by-step simulation.
*   Logic was added to the `on_next_step_button_clicked` function to detect the end of the simulation when all input patterns are processed. Upon completion, the "Next Step" button is disabled, a final status message and cumulative win probability are displayed, and the prediction bar chart is cleared.

### Insights or Next Steps

*   The interactive simulation with step-by-step control and probability visualization is now functional, allowing users to observe the Markov model's learning process and prediction changes at each step.
*   Further development could include adding more detailed visualizations, such as the probability distribution for *all* possible next directions in the bar chart (not just the top N), or allowing the user to input their own patterns during the interactive simulation.


In [42]:
!pip freeze > requirements.txt